In [6]:
import time
import csv
import re
import networkx as nx
import numpy as np

class GraphParser:
    def __init__(self, file_path, dataset_type):
        """
        Initializes the parser with the dataset type and file path.

        :param file_path: Path to the CSV file.
        :param dataset_type: Type of dataset (e.g., "NCAA-FOOTBALL", "DOLPHINS", "LES-MISERABLES").
        """
        self.file_path = file_path
        self.dataset_type = dataset_type

    def clean_numeric_value(self, value):
        """ Strips non-digit characters and converts to integer. """
        return int(re.sub(r'\D', '', value)) if re.search(r'\d', value) else 0

    def parse_csv(self):
        """ Parses the CSV file and returns a list of edges based on dataset type. """
        edges = []
        with open(self.file_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                row = [item.strip() for item in row if item.strip()]  # Remove extra whitespace

                if self.dataset_type == "NCAA-FOOTBALL":
                    self.process_ncaa_football(row, edges)
                elif self.dataset_type == "LES-MISERABLES":
                    self.process_les_miserables(row, edges)
                else:
                    self.process_generic_graph(row, edges)

        return edges

    def process_ncaa_football(self, row, edges):
        """ Processes NCAA-FOOTBALL dataset format. """
        if len(row) < 4:
            return

        team1, score1, team2, score2 = row[:4]
        overtime = row[4] if len(row) > 4 else None

        score1 = self.clean_numeric_value(score1)
        score2 = self.clean_numeric_value(score2)

        # In NCAA-FOOTBALL, the winning team is determined by score.
        # We create an edge from the losing team to the winning team so that the winner gets PageRank credit.
        if score1 > score2:
            edges.append((team2, score2, team1, score1, overtime))
        else:
            edges.append((team1, score1, team2, score2, overtime))

    def process_les_miserables(self, row, edges):
        """ Processes LES-MISERABLES dataset format. """
        if len(row) != 4:
            return

        node1, label, node2, zero_value = row
        label = self.clean_numeric_value(label)
        zero_value = self.clean_numeric_value(zero_value)

        edges.append((node1, label, node2, zero_value))

    def process_generic_graph(self, row, edges):
        """ Processes generic graph datasets (e.g., DOLPHINS, KARATE) """
        if len(row) < 4:
            return

        node1, value1, node2, value2 = row[:4]
        value1 = self.clean_numeric_value(value1)
        value2 = self.clean_numeric_value(value2)

        edges.append((node1, value1, node2, value2))


class PageRank:
    def __init__(self, graph, epsilon=1e-6, d=0.85, max_iter=100):
        """
        Initializes the PageRank instance.
        :param graph: A NetworkX graph (preferably a DiGraph for directed data).
        :param epsilon: Convergence threshold.
        :param d: Damping factor.
        :param max_iter: Maximum number of iterations.
        """
        self.graph = graph
        self.epsilon = epsilon
        self.d = d
        self.max_iter = max_iter
        self.rank = {}  # Dictionary to store the final PageRank scores

    def compute_pagerank_matrix_no_transpose(self):
        """
        Computes PageRank using a matrix-based (vectorized) approach without transposing.
        We build a column-stochastic transition matrix A such that:
           A[i, j] = 1/outdeg(j) if there is an edge from node j to node i.
        Dangling nodes (with no out-links) are handled separately.
        
        :returns: (iterations, rank) where iterations is the number of iterations run and rank is a dictionary of PageRank scores.
        """
        nodes = list(self.graph.nodes())
        N = len(nodes)
        if N == 0:
            return 0, {}
        
        # Create mapping from node to index.
        index = {node: i for i, node in enumerate(nodes)}
        
        # Build a column-stochastic transition matrix A.
        A = np.zeros((N, N))
        dangling = np.zeros(N, dtype=bool)
        for node in nodes:
            j = index[node]  # column index for the source node
            out_deg = self.graph.out_degree(node)
            if out_deg == 0:
                dangling[j] = True
            else:
                for succ in self.graph.successors(node):
                    i = index[succ]
                    A[i, j] = 1.0 / out_deg
        
        # Initialize the PageRank vector uniformly.
        pr = np.ones(N) / N
        iterations = 0
        
        while iterations < self.max_iter:
            old_pr = pr.copy()
            # Compute dangling contribution.
            dangling_sum = old_pr[dangling].sum()
            # Update without transposing (since A is column-stochastic).
            pr = (1 - self.d) / N + self.d * (A.dot(old_pr) + dangling_sum / N)
            # Convergence check using the L1 norm.
            if np.linalg.norm(pr - old_pr, ord=1) < self.epsilon:
                break
            iterations += 1
        
        # Map the PageRank vector back to node labels.
        pr_dict = {node: pr[index[node]] for node in nodes}
        self.rank = pr_dict
        return iterations, pr_dict

In [4]:
parser = GraphParser("data/NCAA_football.csv", "NCAA-FOOTBALL")
edges = parser.parse_csv()
for edge in edges[:10]:  # Print first 10 edges
    print(edge)

('"Northeastern"', 14, 'Ball State', 48, None)
('"UTEP"', 17, 'Buffalo', 42, None)
('"Eastern Illinois"', 12, 'Central Michigan', 31, None)
('"Upper Iowa"', 13, 'Drake', 17, None)
('"Indiana State"', 0, 'Eastern Michigan', 52, None)
('"East Central Oklahoma"', 14, 'Sam Houston State', 58, None)
('"Southwest Baptist"', 28, 'Southeast Missouri State', 35, '"(OT)"')
('"Shorter"', 0, 'Western Carolina', 35, None)
('"Hofstra"', 3, 'Connecticut', 35, None)
('"Jacksonville State"', 14, 'Georgia Tech', 41, None)


In [24]:
if __name__ == "__main__":
    # For testing purposes, we set filename and dataset type.
    filename = "data/NCAA_football.csv"
    dataset_type = "NCAA-FOOTBALL"

    # Time reading/parsing the file and constructing the graph.
    start_read = time.time()
    parser = GraphParser(filename, dataset_type)
    edges = parser.parse_csv()

    # Build a directed graph from the parsed edges.
    # For NCAA-FOOTBALL, each edge is stored as (loser, score, winner, score, overtime).
    G = nx.DiGraph()
    for edge in edges:
        if dataset_type == "NCAA-FOOTBALL":
            winner, _, loser, _, _ = edge
            G.add_edge(winner, loser)#(winner, loser)(loser,winner)
        else:
            node1, _, node2, _ = edge
            G.add_edge(node1, node2)
    end_read = time.time()
    read_time = end_read - start_read

    nx_pr = nx.pagerank(G, alpha=0.95)
    sorted_nx_pr = sorted(nx_pr.items(), key=lambda item: item[1], reverse=True)
    print("NX PageRank:")
    for pos, (node, score) in enumerate(sorted_nx_pr, 1):
        print(f"{pos}: {node} -> {score}")
    

    # Compute PageRank using the matrix-based (transposed) method and time the process.
    pagerank_instance = PageRank(G, epsilon=1e-8, d=0.95, max_iter=100)
    start_process = time.time()
    iterations, pr = pagerank_instance.compute_pagerank_matrix_no_transpose()
    end_process = time.time()
    processing_time = end_process - start_process

    # Sort nodes by PageRank score in descending order and print results.
    sorted_pr = sorted(pr.items(), key=lambda item: item[1], reverse=True)
    for rank_position, (node, score) in enumerate(sorted_pr, 1):
        print(f"{rank_position} {node} with pagerank: {score}")

    print(f"\nRead time: {read_time:.6f} seconds")
    print(f"Processing time: {processing_time:.6f} seconds")
    print(f"Iterations: {iterations}")

NX PageRank:
1: North Dakota -> 0.005297782589965389
2: Weber State -> 0.005235289437575296
3: Montana -> 0.005164984641136441
4: South Dakota -> 0.004862460971611673
5: Northwestern State -> 0.0046059786586773325
6: Florida -> 0.004436726370954163
7: Iona -> 0.0044328205489297815
8: Richmond -> 0.004373365258114208
9: Utah -> 0.004339948780794504
10: Oklahoma -> 0.004269249456878441
11: Central Arkansas -> 0.004256190597382783
12: Sacred Heart -> 0.004206756304488348
13: Savannah State -> 0.0042049809308409005
14: Texas Tech -> 0.004159886440195777
15: Texas -> 0.004117750905629728
16: South Carolina State -> 0.004081730546960439
17: Grambling State -> 0.004081730546960439
18: James Madison -> 0.004054389792789773
19: Bryant University -> 0.004050523423513114
20: Liberty -> 0.003972367530277777
21: Mississippi -> 0.003971933550052845
22: TCU -> 0.003942462347505244
23: Drake -> 0.003923761745085482
24: Samford -> 0.003915555573559509
25: USC -> 0.003898196364562262
26: McNeese State -

In [ ]:
import time
import csv
import re
import networkx as nx
import numpy as np

class GraphParser:
    def __init__(self, file_path, dataset_type):
        self.file_path = file_path
        self.dataset_type = dataset_type

    def clean_numeric_value(self, value):
        return int(re.sub(r'\D', '', value)) if re.search(r'\d', value) else 0

    def parse_csv(self):
        edges = []
        with open(self.file_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                row = [item.strip() for item in row if item.strip()]

                if self.dataset_type == "NCAA-FOOTBALL":
                    self.process_ncaa_football(row, edges)
                elif self.dataset_type == "LES-MISERABLES":
                    self.process_les_miserables(row, edges)
                else:
                    self.process_generic_graph(row, edges)

        return edges

    def process_ncaa_football(self, row, edges):
        if len(row) < 4:
            return

        team1, score1, team2, score2 = row[:4]
        overtime = row[4] if len(row) > 4 else None

        score1 = self.clean_numeric_value(score1)
        score2 = self.clean_numeric_value(score2)
        if score1 > score2:
            edges.append((team2, score2, team1, score1, overtime))
        else:
            edges.append((team1, score1, team2, score2, overtime))

    def process_les_miserables(self, row, edges):
        if len(row) != 4:
            return
        node1, label, node2, zero_value = row
        label = self.clean_numeric_value(label)
        zero_value = self.clean_numeric_value(zero_value)
        edges.append((node1, label, node2, zero_value))

    def process_generic_graph(self, row, edges):
        if len(row) < 4:
            return
        node1, value1, node2, value2 = row[:4]
        value1 = self.clean_numeric_value(value1)
        value2 = self.clean_numeric_value(value2)
        edges.append((node1, value1, node2, value2))

class PageRank:
    def __init__(self, graph, epsilon=1e-8, d=0.85, max_iter=100):
        """
        Initializes PageRank.
        :param graph: A networkx.DiGraph.
        :param epsilon: Convergence threshold.
        :param d: Damping factor.
        :param max_iter: Maximum iterations.
        """
        self.graph = graph
        self.epsilon = epsilon
        self.d = d
        self.max_iter = max_iter
        self.rank = {}

    def compute_pagerank(self):
        """
        A[i, j] = 1 / out_degree(j)   if there is an edge from node j to node i
        returns: (iterations, rank_dict)
        """
        nodes = list(self.graph.nodes())
        N = len(nodes)
        if N == 0:
            return 0, {}
        index = {node: i for i, node in enumerate(nodes)}
        A = np.zeros((N, N))
        dangling = np.zeros(N, dtype=bool)
        for node in nodes:
            j = index[node]
            out_deg = self.graph.out_degree(node)
            if out_deg == 0:
                dangling[j] = True
            else:
                for succ in self.graph.successors(node):
                    i = index[succ]
                    A[i, j] = 1.0 / out_deg

        pr = np.ones(N) / N
        for iteration in range(self.max_iter):
            old_pr = pr.copy()
            dangling_sum = old_pr[dangling].sum()
            pr = (1 - self.d) / N + self.d * (A.dot(old_pr) + dangling_sum / N)
            if np.linalg.norm(pr - old_pr, ord=1) < self.epsilon:
                break

        pr_dict = {node: pr[index[node]] for node in nodes}
        self.rank = pr_dict
        return iteration + 1, pr_dict

if __name__ == "__main__":
    filename = "data/NCAA_football.csv"
    dataset_type = "NCAA-FOOTBALL"
    start_read = time.time()
    parser = GraphParser(filename, dataset_type)
    edges = parser.parse_csv()
    end_read = time.time()
    read_time = end_read - start_read

    G = nx.DiGraph()
    for edge in edges:
        winner, _, loser, _, _ = edge
        G.add_edge(winner, loser)

    start_process = time.time()
    nx_pr = nx.pagerank(G, alpha=0.85)
    end_process = time.time()
    nx_processing_time = end_process - start_process
    sorted_nx_pr = sorted(nx_pr.items(), key=lambda x: x[1], reverse=True)
    unsorted_nx_pr = sorted(nx_pr.items(), key=lambda x: x[1], reverse=False)

    pagerank_instance = PageRank(G, epsilon=1e-10, d=0.85, max_iter=100)
    start_process_custom = time.time()
    iterations_custom, pr_custom = pagerank_instance.compute_pagerank()
    end_process_custom = time.time()
    custom_processing_time = end_process_custom - start_process_custom
    sorted_custom_pr = sorted(pr_custom.items(), key=lambda x: x[1], reverse=True)

    print("=== NetworkX PageRank (top 10) ===")
    for i, (node, score) in enumerate(sorted_nx_pr[:10], 1):
        print(f"{i}. {node} -> {score}")

    print("\n=== Custom PageRank (top 10) ===")
    for i, (node, score) in enumerate(sorted_custom_pr[:10], 1):
        print(f"{i}. {node} -> {score}")

    print(f"\nNumber of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Read time: {read_time:.6f} seconds")
    print(f"NetworkX PageRank processing time: {nx_processing_time:.6f} seconds")
    print(f"Custom PageRank processing time: {custom_processing_time:.6f} seconds")
    print(f"Custom PageRank iterations: {iterations_custom}")


=== NetworkX PageRank (top 10) ===
1. North Dakota -> 0.005045914837189206
2. Weber State -> 0.004987863753108685
3. Montana -> 0.004922556283518098
4. South Dakota -> 0.004641536262855575
5. Northwestern State -> 0.004403284938608435
6. Florida -> 0.0042460632525570235
7. Iona -> 0.004242435059801991
8. Richmond -> 0.004187205903419829
9. Utah -> 0.004156164698737884
10. Oklahoma -> 0.004090490745030627

=== Custom PageRank (top 10) ===
1. North Dakota -> 0.005045134098984914
2. Weber State -> 0.004987096784402064
3. Montana -> 0.004921804805496358
4. South Dakota -> 0.004640851441720289
5. Northwestern State -> 0.004402656629786509
6. Florida -> 0.0042454722361246235
7. Iona -> 0.004241844903963197
8. Richmond -> 0.004186628847728124
9. Utah -> 0.0041555950059025725
10. Oklahoma -> 0.004089936629808844

Number of nodes: 570
Number of edges: 1535
Read time: 0.000000 seconds
NetworkX PageRank processing time: 0.011921 seconds
Custom PageRank processing time: 0.005152 seconds
Custom Pag

In [ ]:
if __name__ == "__main__":
    filename = "data/dolphins.csv"  
    dataset_type = "DOLPHINS"        

    start_read = time.time()
    parser = GraphParser(filename, dataset_type)
    edges = parser.parse_csv()
    end_read = time.time()
    read_time = end_read - start_read
    G = nx.Graph()
    for edge in edges:
        #(dolphin1, val1, dolphin2, val2)
        dolphin1, _, dolphin2, _ = edge
        G.add_edge(dolphin1, dolphin2)
    
    start_process = time.time()
    G_directed = nx.DiGraph()
    for e in G.edges():
        G_directed.add_edge(e[0], e[1])
        G_directed.add_edge(e[1], e[0])
    nx_pr = nx.pagerank(G_directed, alpha=0.85)

    nx_pr = nx.pagerank(G_directed, alpha=0.85)
    end_process = time.time()
    processing_time = end_process - start_process

    pagerank_instance = PageRank(G_directed, epsilon=1e-10, d=0.85, max_iter=100)
    start_process_custom = time.time()
    iterations_custom, pr_custom = pagerank_instance.compute_pagerank()
    end_process_custom = time.time()
    custom_processing_time = end_process_custom - start_process_custom
    sorted_custom_pr = sorted(pr_custom.items(), key=lambda x: x[1], reverse=True)

    print("\n=== Custom PageRank (top 10) ===")
    for i, (node, score) in enumerate(sorted_custom_pr[:10], 1):
        print(f"{i}. {node} -> {score}")
    print(f"Custom PageRank iterations: {iterations_custom}")
    
    sorted_nx_pr = sorted(nx_pr.items(), key=lambda x: x[1], reverse=True)
    for i, (node, score) in enumerate(sorted_nx_pr[:10], 1):
        print(f"{i}. {node}: {score}")

    print(f"\nRead time: {read_time:.6f} seconds")
    print(f"Processing time: {processing_time:.6f} seconds")



=== Custom PageRank (top 10) ===
1. Grin -> 0.032144492767568124
2. Jet -> 0.03172814043060556
3. Trigger -> 0.03129935689421097
4. Web -> 0.03009537140536281
5. SN4 -> 0.029875338593220523
6. Topless -> 0.02951420357096363
7. Scabs -> 0.028423069722867778
8. Patchback -> 0.02645855018299648
9. Gallatin -> 0.02615687599797234
10. Beescratch -> 0.024650716860371423
Custom PageRank iterations: 86
1. Grin: 0.032137029718865714
2. Jet: 0.0317411226125807
3. Trigger: 0.031292342498045814
4. Web: 0.030108941362489344
5. SN4: 0.029868954502794757
6. Topless: 0.02950713720344268
7. Scabs: 0.028416738215896527
8. Patchback: 0.026452435542402356
9. Gallatin: 0.02616992597905726
10. Beescratch: 0.024657779163893456

Read time: 0.002996 seconds
Processing time: 0.004510 seconds


In [ ]:
if __name__ == "__main__":
    filename = "data/lesmis.csv"       
    dataset_type = "LES-MISERABLES"

    start_read = time.time()
    parser = GraphParser(filename, dataset_type)
    edges = parser.parse_csv()
    end_read = time.time()
    read_time = end_read - start_read
    G = nx.Graph()
    for edge in edges:
        char1, label, char2, zero_value = edge
        G.add_edge(char1, char2)
    
    start_process = time.time()
    G_dir = nx.DiGraph()
    for u, v in G.edges():
        G_dir.add_edge(u, v)
        G_dir.add_edge(v, u)

    nx_pr = nx.pagerank(G_dir, alpha=0.85)
    end_process = time.time()
    processing_time = end_process - start_process

    pagerank_instance = PageRank(G_dir, epsilon=1e-10, d=0.85, max_iter=100)
    start_process_custom = time.time()
    iterations_custom, pr_custom = pagerank_instance.compute_pagerank()
    end_process_custom = time.time()
    custom_processing_time = end_process_custom - start_process_custom
    sorted_custom_pr = sorted(pr_custom.items(), key=lambda x: x[1], reverse=True)

    print("\n=== Custom PageRank (top 10) ===")
    for i, (node, score) in enumerate(sorted_custom_pr[:10], 1):
        print(f"{i}. {node} -> {score}")
    print(f"Custom PageRank iterations: {iterations_custom}")

    sorted_nx_pr = sorted(nx_pr.items(), key=lambda x: x[1], reverse=True)
    for i, (node, score) in enumerate(sorted_nx_pr[:10], 1):
        print(f"{i}. {node}: {score}")

    print(f"\nRead time: {read_time:.6f} seconds")
    print(f"Processing time: {processing_time:.6f} seconds")



=== Custom PageRank (top 10) ===
1. Valjean -> 0.07543012164773534
2. Myriel -> 0.04277928105974448
3. Gavroche -> 0.03576731818311261
4. Marius -> 0.030894936207253653
5. Javert -> 0.030302735905698455
6. Thenardier -> 0.027926525691254794
7. Fantine -> 0.02702270491839203
8. Enjolras -> 0.02188203331961119
9. Cosette -> 0.020611215084313916
10. MmeThenardier -> 0.019501134690098298
Custom PageRank iterations: 76
1. Valjean: 0.07543374445332475
2. Myriel: 0.04280343976075732
3. Gavroche: 0.03576412343161933
4. Marius: 0.030892701920406464
5. Javert: 0.03030259712717036
6. Thenardier: 0.027925725316528028
7. Fantine: 0.027022474221419405
8. Enjolras: 0.021879692835635844
9. Cosette: 0.0206109951184114
10. MmeThenardier: 0.019500856219565182

Read time: 0.003340 seconds
Processing time: 0.005623 seconds
